# 지식창고로 함수를 사용하는 방법

이 노트북에서는 지식창고에 액세스할 수 있는 에이전트와 사용자 요구 사항에 따라 호출할 수 있는 두 개의 함수를 만들어 [인수 생성]('How_to_generate_function_arguments_with_chat_models.ipynb') 노트북의 개념을 기반으로 합니다.

학술 주제에 대한 질문에 답하기 위해 arXiv의 데이터를 사용하는 에이전트를 만들어 보겠습니다. 이 에이전트에는 다음과 같은 두 가지 함수가 있습니다: 

- **get_articles:** 주제에 대한 arXiv 논문을 가져와 링크와 함께 사용자에게 요약하는 함수 
- **read_article_and_summarize**: 이 기능은 이전에 검색한 논문 중 하나를 가져와 전체 내용을 읽고 핵심 주장, 증거 및 결론을 요약합니다.

이렇게 하면 여러 서비스 중에서 선택할 수 있고 첫 번째 함수의 데이터 중 일부가 두 번째 함수에서 계속 사용되는 다기능 워크플로우에 익숙해질 수 있습니다.

## 연습

이 쿡북에서는 다음과 같은 워크플로우를 안내합니다:

- **검색 유틸리티:** 답변을 얻기 위해 arXiv에 액세스하는 두 가지 함수 만들기 
- **에이전트 구성:** 함수의 필요성을 평가하고, 필요한 경우 해당 함수를 호출하여 결과를 에이전트에게 다시 제시하는 에이전트 동작 구축 
- **arXiv 대화:** 이 모든 것을 실시간 대화에 통합하기.

In [1]:
!pip install scipy
!pip install tenacity
!pip install tiktoken==0.3.3
!pip install termcolor 
!pip install openai
!pip install requests
!pip install arxiv
!pip install pandas
!pip install PyPDF2
!pip install tqdm

  Using cached tiktoken-0.3.3-cp310-cp310-macosx_11_0_arm64.whl (706 kB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.4.0
    Uninstalling tiktoken-0.4.0:
      Successfully uninstalled tiktoken-0.4.0


In [53]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-16k-0613"
# GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"


## 검색 유틸리티

먼저 두 가지 기능을 뒷받침할 몇 가지 유틸리티를 설정하겠습니다.

다운로드한 논문은 디렉토리에 저장됩니다(여기서는 ```./data/papers```을 사용합니다). 다운로드한 논문의 임베딩과 세부 정보를 저장하여 ```summarize_text```를 사용하여 검색할 수 있도록 ``arxiv_library.csv`` 파일을 생성합니다.

In [54]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)


In [55]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response


def get_articles(query, library=paper_dir_filepath, top_k=5):
    """This function gets the top_k articles based on a user's query, sorted by relevance.
    It also downloads the files and stores them in arxiv_library.csv to be retrieved by the read_article_and_summarize.
    """
    search = arxiv.Search(
        query=query, max_results=top_k, sort_by=arxiv.SortCriterion.Relevance
    )
    result_list = []
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list


In [56]:
# Test that the search is working
result_output = get_articles("ppo reinforcement learning")
result_output[0]


{'title': 'Proximal Policy Optimization and its Dynamic Version for Sequence Generation',
 'summary': 'In sequence generation task, many works use policy gradient for model\noptimization to tackle the intractable backpropagation issue when maximizing\nthe non-differentiable evaluation metrics or fooling the discriminator in\nadversarial learning. In this paper, we replace policy gradient with proximal\npolicy optimization (PPO), which is a proved more efficient reinforcement\nlearning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We\ndemonstrate the efficacy of PPO and PPO-dynamic on conditional sequence\ngeneration tasks including synthetic experiment and chit-chat chatbot. The\nresults show that PPO and PPO-dynamic can beat policy gradient by stability and\nperformance.',
 'article_url': 'http://arxiv.org/abs/1808.07982v1',
 'pdf_url': 'http://arxiv.org/pdf/1808.07982v1'}

In [57]:
result_output

[{'title': 'Proximal Policy Optimization and its Dynamic Version for Sequence Generation',
  'summary': 'In sequence generation task, many works use policy gradient for model\noptimization to tackle the intractable backpropagation issue when maximizing\nthe non-differentiable evaluation metrics or fooling the discriminator in\nadversarial learning. In this paper, we replace policy gradient with proximal\npolicy optimization (PPO), which is a proved more efficient reinforcement\nlearning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We\ndemonstrate the efficacy of PPO and PPO-dynamic on conditional sequence\ngeneration tasks including synthetic experiment and chit-chat chatbot. The\nresults show that PPO and PPO-dynamic can beat policy gradient by stability and\nperformance.',
  'article_url': 'http://arxiv.org/abs/1808.07982v1',
  'pdf_url': 'http://arxiv.org/pdf/1808.07982v1'},
 {'title': 'CIM-PPO:Proximal Policy Optimization with Liu-Correntropy Induced Metric',
  

In [58]:
# Test that the search is working
result_output = get_articles("toolformer")
result_output[0]


{'title': 'Graph-ToolFormer: To Empower LLMs with Graph Reasoning Ability via Prompt Augmented by ChatGPT',
 'summary': 'In this paper, we aim to develop a large language model (LLM) with the\nreasoning ability on complex graph data. Currently, LLMs have achieved very\nimpressive performance on various natural language learning tasks, extensions\nof which have also been applied to study the vision tasks with multi-modal\ndata. However, when it comes to the graph learning tasks, existing LLMs present\nvery serious flaws due to their several inherited weaknesses in performing\n{multi-step logic reasoning}, {precise mathematical calculation} and\n{perception about the spatial and temporal factors}.\n  To address such challenges, in this paper, we will investigate the\nprinciples, methodologies and algorithms to empower existing LLMs with graph\nreasoning ability, which will have tremendous impacts on the current research\nof both LLMs and graph learning. Inspired by the latest ChatGPT and

In [59]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]


In [76]:
def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarize chunk of text"""
    prompt = template_prompt + content
    response = openai.ChatCompletion.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response["choices"][0]["message"]["content"]


def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning.\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary in Korean collated from this collection of key points extracted from an academic paper. Must use Korean.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n""",
            }
        ],
        temperature=0,
    )
    return response


In [61]:
# Test the summarize_text function works
chat_test_response = summarize_text("PPO reinforcement learning sequence generation")


Chunking text from paper
Summarizing each chunk of text


100%|██████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.54s/it]


Summarizing into overall summary


In [62]:
print(chat_test_response["choices"][0]["message"]["content"])


핵심 주장:
- 이 논문은 시퀀스 생성 작업에서 Proximal Policy Optimization (PPO)의 사용에 대해 논의한다.
- PPO-dynamic이라는 수정된 버전의 PPO를 제안하고, 이를 시퀀스 생성 모델 최적화에 일반적으로 사용되는 Policy Gradient와 비교한다.
- 결과는 PPO와 PPO-dynamic이 안정성과 성능 측면에서 Policy Gradient보다 우수함을 보여준다.

증거:
- PPO-dynamic은 합성 계수 작업에서 다른 알고리즘보다 더 높은 정확도를 보여준다.
- PPO-dynamic은 PPO보다 더 빠른 학습 진행을 보여준다.
- OpenSubtitles 데이터셋을 사용한 chit-chat chatbot 작업에서 PPO-dynamic은 다른 알고리즘과 비교되며 BLEU-2 점수를 사용하여 평가된다. 결과는 제공되지 않는다.

결론:
- MLE, REINFORCE 및 PPO 세 가지 최적화 방법은 비슷한 성능을 보이지만, PPO-dynamic은 다른 방법보다 약간 더 높은 BLEU-2 점수를 달성한다.
- PPO와 PPO-dynamic의 학습 진행은 Policy Gradient보다 안정적이며, PPO-dynamic은 더 빠르게 수렴한다.
- 결과는 PPO가 시퀀스 학습에 더 적합하며, REINFORCE 사용 시 발생하는 높은 분산 문제를 개선할 수 있다는 것을 시사한다.
- PPO-dynamic은 최적화 과정을 더 개선한다.

증거:
1. 논문은 두 확률 분포 사이의 Kullback-Leibler (KL) 발산을 계산하는 수학적 식 (식 1)을 제시한다.
2. 논문은 이전 확률 분포와 새로운 확률 분포의 비율로 정의된 상수 β(x)를 소개한다 (식 2).
3. 논문은 식 1의 β(x) 항을 식 3의 식으로 대체하여 KL 발산에 대한 식 (식 4)을 유도한다.
4. 논문은 α(a)가 1보다 작다고 가정하고, ln(1 + α(a)) 및 ln(1 - α(a)Pold(a)/(1 - Pold(a)))를 근사하기 위해 Ta

## 에이전트 구성

이 단계에서는 API를 여러 차례 사용할 수 있도록 지원하는 ```Conversation``` 클래스와 ```ChatCompletion``` API와 지식창고 함수 간의 상호 작용을 지원하는 몇 가지 Python 함수를 포함하여 에이전트를 만들겠습니다.

In [63]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [64]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [65]:
# Initiate our get_articles and read_article_and_summarize functions
arxiv_functions = [
    {
        "name": "get_articles",
        "description": """Use this function to get academic papers from arXiv to answer user questions.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query in JSON. Responses should be summarized and should include the article URL reference
                            """,
                }
            },
            "required": ["query"],
        },
        "name": "read_article_and_summarize",
        "description": """Use this function to read whole papers and provide a summary for users.
        You should NEVER call this function before get_articles has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Description of the article in plain text based on the user's query
                            """,
                }
            },
            "required": ["query"],
        },
    }
]


In [66]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_arxiv_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()


def call_arxiv_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message["message"]["function_call"]["name"] == "get_articles":
        try:
            parsed_output = json.loads(
                full_message["message"]["function_call"]["arguments"]
            )
            print("Getting search results")
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(results),
            }
        )
        try:
            print("Got search results, summarizing content")
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")

    elif (
        full_message["message"]["function_call"]["name"] == "read_article_and_summarize"
    ):
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        print("Finding and reading paper")
        summary = summarize_text(parsed_output["query"])
        return summary

    else:
        raise Exception("Function does not exist and cannot be called")


## arXiv 대화

대화에서 함수를 테스트하여 이 모든 것을 종합해 보겠습니다.

In [67]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [68]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does PPO reinforcement learning work?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|████████████████████████████████████████████████████████████| 21/21 [00:08<00:00,  2.48it/s]


Summarizing into overall summary


핵심 주장:
- 이 논문은 심층 강화 학습의 맥락에서 클리핑된 대체 목적 함수를 사용하는 근접 정책 최적화 알고리즘 (PPO-Clip)의 이론적 분석에 초점을 맞추고 있다.
- PPO-Clip의 전역 수렴 속도 보장을 신경 기반 함수 근사화에 대한 첫 번째 보장으로 제시하고자 한다.
- PPO-Clip의 전역 수렴, 수렴 속도의 명시적 특성화, PPO-Clip의 재해석의 실험을 통한 검증 등이 이 논문의 주요 기여 사항이다.

증거:
- PPO-Clip의 분석에 대한 두 가지 기본적인 도전 과제를 식별하였다.
- 저자들은 PPO-Clip의 클리핑 메커니즘으로 인해 각 정책 업데이트에 대한 단순한 닫힌 형식 표현의 부재와 신경 PPO-Clip의 장점 함수의 근사 오차와 클리핑 행동 간의 강한 결합을 지적하였다.
- 이러한 도전 과제를 해결하기 위해 저자들은 두 가지 핵심 아이디어를 제안하였다.
- 첫째로, 저자들은 PPO-Clip을 힌지 손실의 관점에서 재해석하여 정책 개선을 큰 마진 분류 문제의 해결과 연결하였다.
- 둘째로, 저자들은 Entropic Mirror Descent Algorithm (EMDA)를 활용한 두 단계 정책 개선 체계를 제안하였다.
- 이러한 프레임워크는 PPO-Clip의 정책 업데이트에 대한 다루기 쉬운 표현을 가능하게 하며, 클리핑 함수의 수렴에 대한 명시적인 특성화를 제공한다.

결론:
- 이 논문은 심층 강화 학습에서 PPO-Clip의 성능과 수렴 특성에 대한 이론적 통찰력을 제공하며, 중요한 미해결 질문에 대한 대답을 제시하고 PPO-Clip의 분석과 일반화를 위한 프레임워크를 제공한다.

In [69]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Can you read the PPO sequence generation paper for me and give me a summary",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
display(Markdown(updated_response["choices"][0]["message"]["content"]))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|██████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.16s/it]


Summarizing into overall summary


핵심 주장:
- 이 논문은 시퀀스 생성 작업에서 근접 정책 최적화(PPO)의 사용에 대해 논의한다.
- PPO-dynamic이라는 동적 접근법을 제안하고, 시퀀스 생성 모델 최적화에 일반적으로 사용되는 정책 기울기와의 효과를 비교한다.
- 결과는 PPO와 PPO-dynamic이 안정성과 성능 측면에서 정책 기울기를 능가한다는 것을 보여준다.
- 논문은 또한 chit-chat 챗봇, 강화학습, 정책 기울기와 PPO의 텍스트 생성 작업에서의 사용에 대한 배경 정보를 제공한다.

증거:
- PPO-dynamic은 학습 곡선과 정확도 측면에서 다른 알고리즘보다 우수한 성능을 보인다.
- PPO-dynamic은 REINFORCE와 MIXER와 비교했을 때 비슷한 성능을 보이며, 다양한 답변을 제공하는 능력을 보인다.
- PPO-dynamic은 OpenSubtitles 데이터셋을 사용한 chit-chat 챗봇 작업에서 BLEU-2 점수를 사용하여 테스트되고 평가된다.
- PPO-dynamic은 다른 최적화 알고리즘보다 약간 더 높은 BLEU-2 점수를 달성한다.
- PPO와 PPO-dynamic의 학습 진행은 정책 기울기보다 안정적이며, PPO-dynamic은 빠르게 수렴한다.

결론:
- PPO는 시퀀스 학습에 더 나은 방법이며, REINFORCE를 사용하는 고분산 문제를 개선할 수 있다.
- PPO-dynamic은 최적화 과정을 더욱 개선한다.

논문의 핵심 내용:
- KL 다이버전스를 계산하는 수학적 식(Equation 1)을 제시한다.
- 이전 확률 분포와 새로운 확률 분포의 비율로 정의된 상수 β(x)를 소개한다(Equation 2).
- KL 다이버전스를 계산하는 식(Equation 4)을 유도한다.
- α(a)가 1보다 작다고 가정하고, ln(1 + α(a))와 ln(1 - α(a)Pold(a)/(1 - Pold(a)))를 근사화하는 Taylor 전개를 사용한다(Equation 5).
- Equation 4의 항목을 Equation 5의 식으로 대체하여 KL 다이버전스의 단순화된 식(Equation 6)을 얻는다.
- KL 다이버전스의 유도된 식(Equation 7)에 기반한 α(a)의 값 범위를 제시한다.
- PPO와 PPO-dynamic 알고리즘의 의사 코드를 제공한다(Algorithm 1).
- 기준선의 훈련과 다른 알고리즘에 사용된 하이퍼파라미터를 포함한 실험 설정을 설명한다.
- REINFORCE와 PPO 방법에 의해 생성된 출력 분포를 비교하는 Figure 1을 제시한다.
- REINFORCE와 PPO-dynamic 방법에 의해 생성된 출력 분포의 분산을 비교하는 Table 1을 제시한다.
- REINFORCE, PPO, PPO-dynamic 알고리즘을 사용하여 훈련된 chit-chat 챗봇에 의해 생성된 출력 예시를 제시하는 Table 2를 제시한다.
- 순환 신경망과 근접 정책 최적화 알고리즘에 대한 관련 연구에 대한 참고 자료를 포함한다.

In [70]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'},
 {'role': 'user', 'content': 'Hi, how does PPO reinforcement learning work?'},
 {'role': 'assistant',
  'content': '핵심 주장:\n- 이 논문은 심층 강화 학습의 맥락에서 클리핑된 대체 목적 함수를 사용하는 근접 정책 최적화 알고리즘 (PPO-Clip)의 이론적 분석에 초점을 맞추고 있다.\n- PPO-Clip의 전역 수렴 속도 보장을 신경 기반 함수 근사화에 대한 첫 번째 보장으로 제시하고자 한다.\n- PPO-Clip의 전역 수렴, 수렴 속도의 명시적 특성화, PPO-Clip의 재해석의 실험을 통한 검증 등이 이 논문의 주요 기여 사항이다.\n\n증거:\n- PPO-Clip의 분석에 대한 두 가지 기본적인 도전 과제를 식별하였다.\n- 저자들은 PPO-Clip의 클리핑 메커니즘으로 인해 각 정책 업데이트에 대한 단순한 닫힌 형식 표현의 부재와 신경 PPO-Clip의 장점 함수의 근사 오차와 클리핑 행동 간의 강한 결합을 지적하였다.\n- 이러한 도전 과제를 해결하기 위해 저자들은 두 가지 핵심 아이디어를 제안하였다.\n- 첫째로, 저자들은 PPO-Clip을 힌지 손실의 관점에서 재해석하여 정책 개선을 큰 마진 분류 문제의 해결과 연결하였

In [71]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [72]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'}]

## transformer 논문

In [73]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [74]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'}]

In [75]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does Transformer work?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|████████████████████████████████████████████████████████████| 12/12 [00:06<00:00,  1.83it/s]


Summarizing into overall summary


핵심 주장:
- 이 학술 논문은 Toolformer라는 언어 모델을 소개합니다. 이 모델은 간단한 API를 통해 외부 도구를 사용하는 방법을 스스로 학습할 수 있습니다.
- Toolformer는 최신 정보에 접근하거나 정확한 계산을 수행하는 등 현재 언어 모델의 한계를 극복하기 위해 개발되었습니다.
- Toolformer는 자기 지도 학습 방식으로 훈련되며, 각 API에 대해 몇 가지 데모만 필요합니다.
- Toolformer는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템, 캘린더 등 다양한 도구를 포함하고 있습니다.
- 실험 결과, Toolformer는 다양한 하위 작업에서 큰 모델과 경쟁력 있는 성능을 보여주며, 핵심 언어 모델 능력을 희생하지 않고 큰 모델보다 우수한 성능을 달성합니다.

증거:
- 이 논문은 큰 언어 모델을 사용하여 컨텍스트 학습을 통해 데이터셋을 생성하고 자기 지도 손실을 사용하여 유용한 API 호출을 결정하는 방법에 기반합니다.
- Toolformer는 어떤 도구를 언제, 어떻게 사용할지 결정할 수 있으므로 특정 작업에 제한되지 않고 도구를 포괄적으로 사용할 수 있습니다.
- 실험 결과, Toolformer는 도구 사용을 학습한 후 다양한 작업에서 큰 모델과 다른 기준선 모델보다 우수한 성능을 보입니다.

결론:
- Toolformer는 외부 도구를 사용하는 방법을 스스로 학습하는 언어 모델로, 다양한 작업에서 우수한 성능을 보입니다.
- Toolformer는 현재 언어 모델의 한계를 극복하고 최신 정보에 접근하고 정확한 계산을 수행할 수 있도록 도와줍니다.